# Libraries

In [ ]:
import numpy as np
import tensorflow as tf 
import os 

#keras library 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical
from keras.optimizers import SGD
from klepto.archives import file_archive
from keras.callbacks import ModelCheckpoint

print("Tensorflow version %s" %tf.__version__)

# Reading DB file

In [ ]:
db = file_archive("DBf/weatherData.txt")
db.load();
perval=0.10;
datadicdb=db['datadic']   
l_datadicdb=db['l_datadic'] 
del db;

# Setting Up 

In [ ]:
train_images    =[]
train_labels    =[]
test_images     =[]
test_labels     =[]
for ls_datadicdb in l_datadicdb:
    lencd=len(datadicdb[ls_datadicdb])
    cutld=int(lencd*(1-perval))
    train_images+=( datadicdb[ls_datadicdb][:cutld])
    train_labels+=( l_datadicdb[ls_datadicdb][:cutld])
    test_images +=(datadicdb[ls_datadicdb][cutld+1:lencd])
    test_labels +=( l_datadicdb[ls_datadicdb][cutld+1:lencd])

train_images=np.array(train_images)
train_labels=np.array(train_labels)-1
test_images=np.array(test_images)
test_labels=np.array(test_labels)-1

train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5


train_images3 = train_images
test_images3 = test_images

# Applying CNN

In [ ]:
model = Sequential([
  Conv2D(8, 3, input_shape=(150, 150, 3), use_bias=False),
  MaxPooling2D(pool_size=2),
  Flatten(),
  Dense(4, activation='softmax'),
])

model.compile(SGD(lr=.002), loss='categorical_crossentropy', metrics=['accuracy'])
# checkpoint
filepath="Weather-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint]

model.fit(
  train_images3,
  to_categorical(train_labels),
  batch_size=20,
  epochs=10,
  callbacks=callbacks_list,
  validation_data=(test_images3, to_categorical(test_labels)),
)